In [3]:
import pandas as pd

### Load and Clean data

In [4]:
xls = pd.ExcelFile('donnees_entrepots.xls')
entrepot = pd.read_excel(xls, "Données_détaillées_par_aires")
entrepot.columns = entrepot.iloc[1]
entrepot = entrepot.drop([0,1], axis=0).reset_index(drop=True)

# we keep only the first city where the area is located
entrepot["commune_clean"] = entrepot["Communes concernées par l'aire logistique"].apply(lambda x : x.split(",")[0])

In [23]:
# Reduce dataset to large logistic areas
entrepot_majeur = entrepot[entrepot["Surface totale"] > 75000]

In [6]:
# Localize the cities
from geopy import Nominatim

def add_location(ini_dataset) :
    locator = Nominatim(user_agent="myGeocoder")
    ini_dataset["location"] = ini_dataset["commune_clean"].apply(lambda x: locator.geocode(x)[1])
    ini_dataset["latitude"] = ini_dataset["location"].str[0]
    ini_dataset["longitude"] = ini_dataset["location"].str[1]
    return ini_dataset

In [19]:
entrepot_majeur

1,Identifiant aire logistique dense (e1),Région d'implantation,Numéro aires dans région,Communes concernées par l'aire logistique,Nombre d'EPL de plus de 5 000 m2,Surface totale,P_Transport_et_entreposage,P_commerce,P_industrie,P_autres,Eff_com_entreposage,EFF_EPL_5000,Poids de l'entreposage,Chargement,Déchargement,commune_clean
0,1,52,3,"Saint-Nazaire, Montoir-de-Bretagne, Trignac",10 - 19,155000,0.6,0.1,0.3,0,1800,300,0.045253,30,30,Saint-Nazaire
2,12,24,4,"Parçay-Meslay, Tours",3 - 9,142000,0.444444,0.333333,0.222222,0,NC,NC,NC,NC,NC,Parçay-Meslay
4,20,52,2,"Saint-Barthélemy-d'Anjou, Angers, Écouflant",10 - 19,247000,0.529412,0.352941,0.058824,0.058824,1900,300,0.022602,30,30,Saint-Barthélemy-d'Anjou
5,23,28,7,"Rogerville, Oudalle, Gonfreville-l'Orcher",3 - 9,166000,1,0,0,0,NC,NC,NC,NC,NC,Rogerville
7,28,32,19,"Dunkerque, Coudekerque-Branche",3 - 9,144000,0.166667,0.5,0.333333,0,NC,NC,NC,NC,NC,Dunkerque
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,5611,11,42,Vert-Saint-Denis,3 - 9,171000,1,0,0,0,NC,NC,NC,NC,NC,Vert-Saint-Denis
366,5934,44,30,"Thierville-sur-Meuse, Verdun",3 - 9,113000,0.25,0.5,0,0.25,NC,NC,NC,NC,NC,Thierville-sur-Meuse
369,5999,11,23,Brie-Comte-Robert,3 - 9,110000,0.6,0.4,0,0,NC,NC,NC,NC,NC,Brie-Comte-Robert
376,6630,84,31,Reyrieux,3 - 9,83000,0.75,0,0.25,0,NC,NC,NC,NC,NC,Reyrieux


In [10]:
entrepot_majeur = add_location(entrepot_majeur)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Saint-%C3%89tienne-du-Rouvray&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [106]:
#locator.geocode("Boulogne sur mer 62200")

Location(Dannes, Boulogne-sur-Mer, Pas-de-Calais, Hauts-de-France, France métropolitaine, 62200, France, (50.59368681860731, 1.583020224543379, 0.0))

In [16]:
final = pd.read_csv("../final_all_logistic_areas.csv")

In [24]:
entrepot_majeur = entrepot_majeur[["Identifiant aire logistique dense (e1)", "Surface totale", "Région d'implantation"]].merge(final[["Identifiant aire logistique dense (e1)", "location", "longitude", "latitude"]], on="Identifiant aire logistique dense (e1)", how='left')

In [28]:
entrepot_majeur

,Identifiant aire logistique dense (e1),Surface totale,Région d'implantation,location,longitude,latitude
0,1,155000,52,"(47.2733517, -2.2138905)",-2.213891,47.273352
1,12,142000,24,"(47.4413885, 0.7456752)",0.745675,47.441389
2,20,247000,52,"(47.4683081, -0.4932941)",-0.493294,47.468308
3,23,166000,28,"(49.5032616, 0.2631791)",0.263179,49.503262
4,28,144000,32,"(51.0347708, 2.3772525)",2.377252,51.034771
...,...,...,...,...,...,...
184,5611,171000,11,"(48.5671024, 2.6252499)",2.625250,48.567102
185,5934,113000,44,"(49.1706321, 5.348031)",5.348031,49.170632
186,5999,110000,11,"(48.6904522, 2.6166737)",2.616674,48.690452
187,6630,83000,84,"(45.9357535, 4.8220491)",4.822049,45.935753


In [27]:
entrepot_majeur["location"] = entrepot_majeur.apply(lambda x : (x["latitude"], x["longitude"]), axis=1)

### K Means

In [22]:
from sklearn.cluster import KMeans
import geopy.distance
def run_kmeans(
    data,
    threshold_km=20,
    threshold_nb_areas_by_centroid=5,
) :

    data = data[["Identifiant aire logistique dense (e1)", "Région d'implantation", "location", "longitude", "latitude"]]
    region_list = list(data["Région d'implantation"].unique())

    final_dataset = pd.DataFrame()
    centroid_by_region = {}

    #Computing centroids region by region
    for region in region_list :
        dataset = data[data["Région d'implantation"] == region]
        
        # initialising limiting factors for KMeans
        areas_not_served = len(dataset)         
        nb_areas_by_centroid = len(dataset)
        
        # set the initial number of clusters at sufficient amount to satisfy max number of areas by station
        i = max(int(areas_not_served / threshold_nb_areas_by_centroid)+1, 1) 
        
        # if the region has only 1 area, run KMeans with 1 centroid
        threshold_not_served = int(0.2*len(dataset)) + 1 if len(dataset)>1 else 0
        
        while (areas_not_served > threshold_not_served) | (nb_areas_by_centroid > threshold_nb_areas_by_centroid) :
            kmeans = KMeans(
                init="random",
                n_clusters=i,
                n_init=10,
                max_iter=300,
                random_state=42
            )

            kmeans.fit(dataset[["latitude", "longitude"]])

            # Store the centroid information for each area
            dataset["centroid"] = kmeans.labels_
            dataset["centroid_coord"] = dataset["centroid"].apply(lambda x : kmeans.cluster_centers_[x])
            dataset["distance_to_centroid"] = dataset.apply(lambda x : geopy.distance.geodesic(x["location"], kmeans.cluster_centers_[x["centroid"]]).km, axis=1)
            
            # Increase nb of centroids by 1
            i += 1

            # Recompute limiting factors
            areas_not_served = sum(dataset["distance_to_centroid"]>threshold_km)
            nb_areas_by_centroid = max(dataset.groupby("centroid")["Identifiant aire logistique dense (e1)"].count().reset_index()["Identifiant aire logistique dense (e1)"])
        
        # Store the centroid information aside
        centroid_by_region[region] = kmeans.cluster_centers_
        
        # Build the final dataset with all information
        final_dataset = pd.concat([final_dataset, dataset], axis=0)

    return final_dataset, centroid_by_region

In [29]:
final_dataset, centroid_by_region = run_kmeans(entrepot_majeur)

/var/folders/rz/99ygd4xd4571c60xq4jqrmm40000gp/T/ipykernel_14359/1566134382.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["centroid"] = kmeans.labels_
/var/folders/rz/99ygd4xd4571c60xq4jqrmm40000gp/T/ipykernel_14359/1566134382.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["centroid_coord"] = dataset["centroid"].apply(lambda x : kmeans.cluster_centers_[x])
/var/folders/rz/99ygd4xd4571c60xq4jqrmm40000gp/T/ipykernel_14359/1566134382.py:43: SettingWithCopyWarning: 
A value is tryin

In [8]:
#final_dataset.to_csv("final_moderate.csv")

NameError: name 'final_dataset' is not defined

In [ ]:
count_by_region = centroids_per_region.groupby("region").agg({"coord": 'count'}).reset_index()

In [57]:
centroids = pd.read_csv("../centroids_coord_all_areas.csv")

In [50]:
centroids = centroids["coord"]

In [60]:
test = centroids["coord"].apply(lambda x:x.split())

In [43]:
final_dataset = pd.read_csv("../final_all_logistic_areas.csv")

### Display output of algo

In [30]:
import numpy as np
centroids = np.concatenate(list(centroid_by_region.values()))

In [48]:
print("Number of centroids : ", len(centroids))

Number of centroids :  183


In [61]:
import plotly.express as px

fig = px.scatter_mapbox(final_dataset, 
                        lat="latitude", 
                        lon="longitude", 
                        zoom=8, 
                        height=800,
                        width=800)

fig.add_scattermapbox(lat = centroids[:,0]
                      ,lon = centroids[:,1]
                      ,marker_size = 6
                      ,marker_color = 'red'
#                       ,marker_symbol = 'star'
                      ,showlegend = True
                     )

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

KeyError: 'key of type tuple not found and not a MultiIndex'

In [33]:
# Number of areas by centroid
nb_areas_by_centroid = final_dataset.groupby(["Région d'implantation", "centroid"])["Identifiant aire logistique dense (e1)"].count().reset_index()

nb_areas_by_centroid["station_size"] = nb_areas_by_centroid["Identifiant aire logistique dense (e1)"].apply(lambda x : (x > 2) + (x > 4))

In [34]:
nb_areas_by_centroid.rename(columns = {"Identifiant aire logistique dense (e1)" : "area_count"}, inplace=True)

In [25]:
#nb_areas_by_centroid.to_csv("station_size_all_areas.csv")

In [35]:
# Centroid coord by region
centroids_df = pd.DataFrame(
    [(k, i) for k, v in centroid_by_region.items() for i in v], 
    columns=['region', 'coord']
)

In [27]:
#centroids_df.to_csv("centroids_coord_all_areas.csv")

In [63]:
centroids_df = pd.read_csv("../centroids_coord_all_areas.csv")

In [64]:
count_by_region = centroids_df.groupby("region").agg({"coord": 'count'}).reset_index()

In [65]:
region = pd.read_excel(xls, "Données régionales")
region.columns = region.iloc[1]
region = region.drop([0,1], axis=0).reset_index(drop=True)

In [66]:
region = region.iloc[:12,:]

In [67]:
count_by_region["region"] = count_by_region["region"].astype(str)

In [68]:
count_by_region = count_by_region.merge(region[["Code région", "Libellé des régions"]], left_on='region', right_on="Code région", how="left")

In [69]:
count_by_region.sort_values(by="coord", ascending=False)

,region,coord,Code région,Libellé des régions
4,32,31,32,Hauts-de-France
5,44,22,44,Grand-Est
10,84,22,84,Auvergne-Rhône-Alpes
0,11,21,11,Île-de-France
1,24,16,24,Centre-Val de Loire
8,75,14,75,Nouvelle Aquitaine
2,27,12,27,Bourgogne-Franche-Comté
7,53,11,53,Bretagne
3,28,10,28,Normandie
6,52,10,52,Pays de la Loire


In [70]:
count_by_region["coord"].sum()

183